In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow_hub as hub #contains a library of pre-trained models which includes the pre trained BERT model that we are going to fine tune
import tensorflow_text as text #this is a module which allows text to be converted into a format that is readable by the BERT model through pre-processing
import tensorflow as tenflow #the main tensorflow module used adding layers to the model 
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from matplotlib import pyplot as plt
import seaborn as sn

def embedSentence(data):
    textProcess = bertText(data)
    return bertEncode(textProcess)['pooled_output']

workEmail = ["Hi Rafferty, An Employee Appraisal has been submitted for you. Please perform the following steps to review and acknowledge this appraisal: Open your browser to our secured portal: https://ultraframe.co.uk/employee-appraisal Input your username, email and password. More info will be sent to your inbox to move forward. Locate the ‘Employee Acknowledgement’ section. Please acknowledge the appraisal by selecting one of the three options in the Employee Acknowledgement section. Select the ‘Save’ button. Please acknowledge your approval by August 01, 2024. Thank you, Management Team","We suspect a security breach happened earlier this week. In order to prevent further damage, we need everyone to change their password immediately. Please click here to do that: Change Password Please do this right away. Thanks! Sincerely, IT"]
enron = pd.read_csv('/kaggle/input/enron-email-dataset/emails.csv', encoding = "ISO-8859-1")
phishset = pd.read_csv('/kaggle/input/phishingemails/Phishing_Email.csv', encoding = "ISO-8859-1")
phishset.drop(columns=['Unnamed: 0'], inplace = True)
enron.drop(columns=['file'], inplace = True)
enron.insert(1, "Email Type", "Safe Email", True)
enron.rename(columns = {"message" : "Email Text"}, inplace = True)
phishset = pd.concat([phishset, enron])
phishsetSafe = phishset[phishset['Email Type'] == 'Safe Email']
phishsetFish = phishset[phishset['Email Type'] == 'Phishing Email']
phishsetSafeBalanced = phishsetSafe.sample(phishsetFish.shape[0])
balancedFish = pd.concat([phishsetSafeBalanced, phishsetFish])
balancedFish['phish'] = balancedFish['Email Type'].apply(lambda x:1 if x=='Phishing Email' else 0)
balancedFish.fillna(value = '', inplace=True)
Training1, Testing1, Training2, Testing2 = train_test_split(balancedFish['Email Text'], balancedFish['phish'],test_size = 0.3, stratify = balancedFish['phish'])
Testing1, Validation1, Testing2, Validation2 = train_test_split(Testing1, Testing2, test_size = 0.33)
bertText = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bertEncode = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
Input = tenflow.keras.layers.Input(shape=(), dtype=tenflow.string, name='text')
processInput = bertText(Input)
Output = bertEncode(processInput)
dropout = tenflow.keras.layers.Dropout(0.1, name="dropout")(Output['pooled_output'])
dropout = tenflow.keras.layers.Dense(1, activation='sigmoid', name="output")(dropout)
fishRonModel = tenflow.keras.Model(inputs=Input,outputs = dropout)
modelMetrics = [tenflow.keras.metrics.BinaryAccuracy(name='accuracy'), tenflow.keras.metrics.Precision(name='precision'), tenflow.keras.metrics.Recall(name='recall')]
fishRonModel.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = modelMetrics)
fishRonModel.fit(Training1,Training2,epochs=1,validation_data= (Validation1, Validation2))
print("Work Emails")
workEmailPred = fishRonModel.predict(workEmail)
print(workEmailPred)






